In [17]:
%pip install -r requirements.txt
#pip installs a list of libraries
#list can be found in requirements.txt

#important import names
#import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
#import seaborn as sns

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\avery\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.classify import ClassifierI
nltk.download('punkt_tab')
nltk.download('tagsets')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\avery\AppData\Roaming\nltk_data...
[nltk_data]   Package average

In [19]:
df_elife_train = pd.read_parquet('data/Elife/train-00000-of-00001.parquet')
#get df for development data

In [20]:
df_elife_train.head(5)
#check data exists

,article,summary,section_headings,keywords,year,title
0,"In temperate climates , winter deaths exceed s...","In the USA , more deaths happen in the winter ...","[Abstract, Introduction, Results, Discussion, ...","[epidemiology, and, global, health]",2018,National and regional seasonal dynamics of all...
1,Whether complement dysregulation directly cont...,Most people have likely experienced the discom...,"[Abstract, Introduction, Results, Discussion, ...","[microbiology, and, infectious, disease, immun...",2019,Complement and CD4+ T cells drive context-spec...
2,Variation in the presentation of hereditary im...,The immune system protects an individual from ...,"[Abstract, Introduction, Results, Discussion, ...","[microbiology, and, infectious, disease, immun...",2015,Phenotypic complementation of genetic immunode...
3,Rapid and flexible interpretation of conflicti...,The brain adapts to control our behavior in di...,"[Abstract, Introduction, Results, Discussion, ...",[neuroscience],2016,Cascade of neural processing orchestrates cogn...
4,Myosin 5a is a dual-headed molecular motor tha...,Cells use motor proteins that to move organell...,"[Abstract, Introduction, Results, Discussion, ...","[structural, biology, and, molecular, biophysics]",2015,Structural dynamics of myosin 5 during process...


In [21]:
og_summ_list = df_elife_train.summary.tolist()
#grab all summaries and put them in a list

In [22]:
test_summ = og_summ_list[28]
#genomes, DNA
#target words for summary #28

In [23]:
term_dict = {"genome" : [("complete set of genetic instructions", "NN"), ("complete sets of genetic instructions", "NNS")]}
#sample dictionairy in the form of key:lemma value:list of tuples where each tuples is a pair of layterm and POS tag

In [24]:
test_summ[:10]
#test data present -> genome/s

'Our genome'

In [ ]:
def create_injections(doc):
    """A function to inject layterms. Takes in a summary and Creates parallel documents, one in raw form and 
    one containing the lemmatized version of each token"""
    lemma_doc = []
    raw_doc =[]
    doc_sents = nltk.sent_tokenize(doc)
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    for sent in doc_sents:
        #print(type(sent))
        doc = regex_token.tokenize(sent)
        doc = [word.lower() for word in doc]
        doc = pos_tag(doc)
        lemmatizer = WordNetLemmatizer() 
        doc2 = [(lemmatizer.lemmatize(word[0]), word[1]) for word in doc]
        lemma_doc.append(doc2)
        raw_doc.append(doc)
    #print(len(raw_doc))
    #print(len(lemma_doc))
    #print(raw_doc)
    #print(lemma_doc)
    #doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    #lemmatizer = WordNetLemmatizer() 
    #doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return raw_doc, lemma_doc

In [13]:
raw, lemma = create_injections(test_summ)
#check that raw data and lemmatized data are of same lenght, sentence tokenizaiton is stable

22
22
[[('our', 'PRP$'), ('genomes', 'NNS'), ('contain', 'VBP'), ('around', 'IN'), ('different', 'JJ'), ('genes', 'NNS'), ('that', 'WDT'), ('code', 'VBP'), ('for', 'IN'), ('instructions', 'NNS'), ('to', 'TO'), ('create', 'VB'), ('proteins', 'NNS'), ('and', 'CC'), ('other', 'JJ'), ('important', 'JJ'), ('molecules', 'NNS')], [('when', 'WRB'), ('changes', 'NNS'), ('or', 'CC'), ('mutations', 'NNS'), ('occur', 'VBP'), ('within', 'IN'), ('these', 'DT'), ('genes', 'NNS'), ('malfunctioning', 'VBG'), ('proteins', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('damaging', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('cell', 'NN'), ('may', 'MD'), ('be', 'VB'), ('produced', 'VBN')], [('researchers', 'NNS'), ('of', 'IN'), ('human', 'JJ'), ('genetics', 'NNS'), ('have', 'VBP'), ('tried', 'VBN'), ('to', 'TO'), ('spot', 'VB'), ('the', 'DT'), ('genetic', 'JJ'), ('mutations', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('associated', 'VBN'), ('with', 'IN'), ('illnesses', 'NNS'), ('for', 'IN'), ('example', 'NN'), ('hear

In [12]:
#This was built to prototype the find function. I'm leaving it here for clarity now but will delete later
for i in range(0, len(lemma)): #for every sentence
    for j in range(0, len(lemma[i])): # for every word in every sentence
        #print(lemma[i][j])
        if lemma[i][j][0] in term_dict: #if word in term_dict
            #print(lemma[i][j][0])
            dict_list = term_dict[lemma[i][j][0]] #get the list of values
            for k in dict_list: #loop through list
                if lemma[i][j][1] == k[1]: #if term found print its location in both docs
                    print("sentence index: ", i, "word index: ", j, k)
                    #word at S0W1 = genomes
                    #word at S3W26 = genome
                    #word at S12W9 = genome
                    #S12W22 = genome
                    #S21W17 = genome

sentence index:  0 word index:  1 ('complete sets of genetic instructions', 'NNS')
sentence index:  3 word index:  26 ('complete set of genetic instructions', 'NN')
sentence index:  12 word index:  9 ('complete set of genetic instructions', 'NN')
sentence index:  12 word index:  22 ('complete set of genetic instructions', 'NN')
sentence index:  21 word index:  17 ('complete set of genetic instructions', 'NN')


In [27]:
def find(lemma):
    """function for matching the location in the raw doc with contents of the lemma doc at the parallel location"""
    collect = []
    for sent_id in range(0, len(lemma)): #for every sentence
        for word_id in range(0, len(lemma[sent_id])): # for every word in every sentence
            #print(lemma[i][j])
            if lemma[sent_id][word_id][0] in term_dict: #if word in term_dict
                #print(lemma[i][j][0])
                dict_list = term_dict[lemma[sent_id][word_id][0]] #get the list of values
                for layterm_tag in dict_list: #loop through list
                    #print(k[1])
                    #print("GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG")
                    if lemma[sent_id][word_id][1] == layterm_tag[1]:
                        collect.append((sent_id, word_id, layterm_tag[0], layterm_tag[1]))
    return collect
                        #print("sentence index: ", i, "word index: ", j, k[0])

In [28]:
rep_list = find(lemma)
print(rep_list)

[(0, 1, 'complete sets of genetic instructions', 'NNS'), (3, 26, 'complete set of genetic instructions', 'NN'), (12, 9, 'complete set of genetic instructions', 'NN'), (12, 22, 'complete set of genetic instructions', 'NN'), (21, 17, 'complete set of genetic instructions', 'NN')]


In [15]:
def replace(rep_list, raw):
    """function to replace med term with layterm"""
    for loc in rep_list:
        sent_ind = loc[0]
        word_ind = loc[1]
        layt = loc[2]
        layt_tag = loc[3]
        raw[sent_ind][word_ind] = (layt, layt_tag)
    

In [16]:
check = replace(rep_list, raw)

In [17]:
print(raw)

[[('our', 'PRP$'), ('complete sets of genetic instructions', 'NNS'), ('contain', 'VBP'), ('around', 'IN'), ('different', 'JJ'), ('genes', 'NNS'), ('that', 'WDT'), ('code', 'VBP'), ('for', 'IN'), ('instructions', 'NNS'), ('to', 'TO'), ('create', 'VB'), ('proteins', 'NNS'), ('and', 'CC'), ('other', 'JJ'), ('important', 'JJ'), ('molecules', 'NNS')], [('when', 'WRB'), ('changes', 'NNS'), ('or', 'CC'), ('mutations', 'NNS'), ('occur', 'VBP'), ('within', 'IN'), ('these', 'DT'), ('genes', 'NNS'), ('malfunctioning', 'VBG'), ('proteins', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('damaging', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('cell', 'NN'), ('may', 'MD'), ('be', 'VB'), ('produced', 'VBN')], [('researchers', 'NNS'), ('of', 'IN'), ('human', 'JJ'), ('genetics', 'NNS'), ('have', 'VBP'), ('tried', 'VBN'), ('to', 'TO'), ('spot', 'VB'), ('the', 'DT'), ('genetic', 'JJ'), ('mutations', 'NNS'), ('that', 'WDT'), ('are', 'VBP'), ('associated', 'VBN'), ('with', 'IN'), ('illnesses', 'NNS'), ('for', 'IN'), (

So the goal of this code is to do a big find and replace. We create two parallel documents, one thats the "raw" document that contains teh article split by sentence then word, where each word is tagged with its POS. The 'lemma" doc is split the same but instead of using words we use lemmas. This allows us to do a two part check to make sure we're inputing the right word in the right location, by matching on both the lemma and POS. This allows for more accurate tagging and avoids issues with homographs. Currently this code is designed to run on one single article. It needs to be optimized to run on some kind of data structure full of articles and maybe wrapped into a single function. the out put of that function needs to be a real readable summary, currently it spits out the separated tokenized pos tagged version. This needs to be stripped and joined.

In [25]:
# What if we DON'T split by sentence?

def make_lemma_doc(og_doc):
    doc = og_doc.split()
    doc = [word.lower() for word in doc]
    pos_doc = pos_tag(doc)
    lemmatizer = WordNetLemmatizer() 
    lemmas = [(lemmatizer.lemmatize(word[0]), word[1]) for word in pos_doc]
    return doc, lemmas


In [26]:
# WHat if we did find+replace at the same time instead of having two methods?
def find_and_replace(raw, lemma):
    """function for matching the location in the raw doc with contents of the lemma doc at the parallel location"""
    for word_id in range(0, len(lemma)): # For every word in the document
        curr_lemma_pos = lemma[word_id] # Get the current lemma,POS pair
        curr_lemma = curr_lemma_pos[0] # Get the current word's lemma
        if curr_lemma in term_dict: # If the lemma is in the dictionary
            replace_options = term_dict[curr_lemma] # Get the list of replacement options
            for layterm_tag in replace_options: # For every layterm, POS pair in the options
                if curr_lemma_pos[1] == layterm_tag[1]: # If the current POS matches the layterm POS
                    raw[word_id] = layterm_tag[0] # Replace the word in the raw doc with the new replacement

In [27]:
summ = test_summ
raw2, lemma = make_lemma_doc(summ) # Make raw and lemma versions of the summary



In [ ]:
# MAIN LOOP
data = df_elife_train # THis is the dataframe
og_summ_list = df_elife_train.summary.tolist()
og_summ_list = [og_summ_list[28]] # For now just have one document to test with
layterm_summaries = [] # Collect layterm summaries 

for summ in og_summ_list: # For every summary in the data (currently elife_train, can be changed above)
    raw, lemma = make_lemma_doc(summ) # Make raw and lemma versions of the summary
    find_and_replace(raw, lemma)
    layterm_summaries.append(raw) # Add the new summary to the list

print(layterm_summaries)

# Save the summaries as a new column in thedataframe
#data['tfidf_summary'] = layterm_summaries 

[['our', 'complete sets of genetic instructions', 'contain', 'around', '20', ',', '000', 'different', 'genes', 'that', 'code', 'for', 'instructions', 'to', 'create', 'proteins', 'and', 'other', 'important', 'molecules', '.', 'when', 'changes', ',', 'or', 'mutations', ',', 'occur', 'within', 'these', 'genes', ',', 'malfunctioning', 'proteins', 'that', 'are', 'damaging', 'to', 'the', 'cell', 'may', 'be', 'produced', '.', 'researchers', 'of', 'human', 'genetics', 'have', 'tried', 'to', 'spot', 'the', 'genetic', 'mutations', 'that', 'are', 'associated', 'with', 'illnesses', ',', 'for', 'example', 'heart', 'diseases', '.', 'however', ',', 'they', 'found', 'that', 'most', 'of', 'these', 'mutations', 'are', 'actually', 'located', 'outside', 'of', 'genes', ',', 'in', 'the', '‘non-coding’', 'areas', 'that', 'make', 'up', 'the', 'majority', 'of', 'our', 'complete set of genetic instructions', '.', 'these', 'mutations', 'do', 'not', 'modify', 'proteins', 'directly', ',', 'which', 'makes', 'it', '

Below is a sample method for skipping words if they are already defined in the summary, using lists of key phrases. - Avery

In [ ]:
# MAKE FUNCTION FOR SKIPPING PHRASES
after_skip_phrases = [["also", "known", "as"]] # List of phrases after the word that makes it skipable 
prev_skip_phrases = [["known", "as"]] # List of phrases BEFORE the word that makes it skipable

max_after_skip = 3 # Longest length of an after_skip phrase
max_prev_skip = 3 # Longest length of a before_skip phrase

def skip(raw, index):
    skip = False # Set skip to false at first
    # Check after words

    after_phrase = [] # To save the following phrase
    for i in range(1, max_after_skip + 1): # Look at next words
        new_index = index + i # Index of the neighbor
        if new_index < len(raw): # Make sure inex in range
            after_phrase.append(raw[new_index])
            if after_phrase in after_skip_phrases: # If it's a skip phrase
                skip = True # Change skip to true
                break # Leave the loop
        else: # If the index is out of range then so will the next one
            break 
    
    if skip == True: # If the skip is true already then there's no point in continuing
        return skip
    
    prev_phrase = [] # TO save the previous phrase
    for i in range(1, max_prev_skip + 1): # Look at previous words
        new_index = index - i 
        if new_index > -1:
            prev_phrase = [raw[index-i]] + prev_phrase # Update the prev phrase
            if prev_phrase in prev_skip_phrases:
                skip = True
                break
        else:
            break

    return skip 

The next cell is the same find/replace method but with a test statement for skip()

In [ ]:
# NEW FIND/REPLACE WITH SKIP
def find_and_replace(raw, lemma):
    """function for matching the location in the raw doc with contents of the lemma doc at the parallel location"""
    for word_id in range(0, len(lemma)): # For every word in the document
        if skip(raw, word_id) == False: # If not a skip word
            curr_lemma_pos = lemma[word_id] # Get the current lemma,POS pair
            curr_lemma = curr_lemma_pos[0] # Get the current word's lemma
            if curr_lemma in term_dict: # If the lemma is in the dictionary
                replace_options = term_dict[curr_lemma] # Get the list of replacement options
                for layterm_tag in replace_options: # For every layterm, POS pair in the options
                    if curr_lemma_pos[1] == layterm_tag[1]: # If the current POS matches the layterm POS
                        raw[word_id] = layterm_tag[0] # Replace the word in the raw doc with the new replacement

The next cell just tests the skip method by itself.

In [11]:
test_raw = ["this", "test", "also", "known", "as"]
result = skip(test_raw, 1)
print(result)

test_raw = ["this", "test", "is", "false"]
result = skip(test_raw, 1)
print(result)


True
False


Below is the same main training loop but testing summaries with skip_phrases. 

In [32]:
# MAIN LOOP
og_summ_list = ["The genomes also known as genetic instructions are important for survival . Genomes help our bodies in many ways .", "Next, we discuss genomes . The genetic instructions known as genomes help our bodies ."] # Test the new skip method
layterm_summaries = [] # Collect layterm summaries 

for summ in og_summ_list: # For every summary in the data (currently elife_train, can be changed above)
    raw, lemma = make_lemma_doc(summ) # Make raw and lemma versions of the summary
    find_and_replace(raw, lemma)
    layterm_summaries.append(raw) # Add the new summary to the list

print(layterm_summaries)

# Save the summaries as a new column in thedataframe
#data['tfidf_summary'] = layterm_summaries 

[['the', 'genomes', 'also', 'known', 'as', 'genetic', 'instructions', 'are', 'important', 'for', 'survival', '.', 'complete sets of genetic instructions', 'help', 'our', 'bodies', 'in', 'many', 'ways', '.'], ['next,', 'we', 'discuss', 'complete sets of genetic instructions', '.', 'the', 'genetic', 'instructions', 'known', 'as', 'genomes', 'help', 'our', 'bodies', '.']]
